In [2]:
import os
import logging
import boto3
import zipfile
import json
import io
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO

In [3]:
s3_host = 'https://obj.umiacs.umd.edu'
access_key_id = "xxxxx"
secret_access_key = "xxxxx"

s3 = boto3.client('s3',
                  endpoint_url=s3_host,
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key)

buntain = 'buntain'


In [4]:
def process(folder_prefix, hashtag_dict):
    response = s3.list_objects_v2(Bucket=buntain, Prefix=folder_prefix)
    i = 0
    files = []
    for obj in response.get('Contents', []):
        i += 1
        object_key = obj['Key']
        if object_key.endswith('.zip'):
            files.append(object_key)
    print(f"number of files: {i}")
    user_count_dict = dict()
    for file in files:
        zip_object = s3.get_object(Bucket=buntain, Key=file)
        zip_contents = zip_object['Body'].read()
        zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
        for file_info in zip_file.infolist():
            with zip_file.open(file_info) as json_file:
                file_name = file_info.filename
                if not file_name.endswith("-tweet.json"):
                    continue
                try:
                    json_data = json_file.read().decode('utf-8')
                except:
                    print("this is a text file")
                parsed_data = json.loads(json_data)
                for ind_data in parsed_data:
                    if 'hashtags' in ind_data['tweet'].keys():
                        for hashtag in ind_data['tweet']['hashtags']:
                            if hashtag in hashtag_dict:
                                hashtag_dict[hashtag] += 1
                            else:
                                hashtag_dict[hashtag] = 1
    return hashtag_dict


In [6]:
folder_prefix_lst = ['twitter.tmrc/August_2022/TMRC14_APAC_1/', 'twitter.tmrc/August_2022/TMRC14_APAC_2/', 'twitter.tmrc/October_2022/TMRC15_APAC_3/']
dict_lst = []
for folder in folder_prefix_lst:
    print(folder)
    hashtag_dict = dict()
    user_dict = process(folder, hashtag_dict)
    dict_lst.append(hashtag_dict)


twitter.tmrc/August_2022/TMRC14_APAC_1/
number of files: 1000
twitter.tmrc/August_2022/TMRC14_APAC_2/
number of files: 1000
twitter.tmrc/October_2022/TMRC15_APAC_3/
number of files: 1000


In [7]:
sorted_dict_lst = []
for d in dict_lst:
    sorted_dict_lst.append(dict(sorted(d.items(), key=lambda item: item[1], reverse=True)))

In [8]:
for d in sorted_dict_lst:
    print(d)

{'papua': 230, 'MenolakLupa': 198, 'IndonesiaNegaraHukum': 165, 'SekilasInfo': 163, 'KKB': 159, 'BersatuPedulidanBerbagi': 146, 'BinmasNokenPolri': 142, 'TumpasKKBPapua': 128, 'BongkarBiangRusuh': 118, 'Covid_19': 117, 'PalestinaMenang': 117, 'DemoCovidMenanti': 102, 'StopHoax': 98, 'BelaNegaraBukanRiziq': 98, 'PolriEdukasiProkes': 90, 'MenujuKekebalanKomunal': 85, '70thHumasPolri': 85, 'PAPUA': 84, 'stophoax': 81, 'FPILaskarHoax': 79, 'PolriGelorakanVaksinasi': 79, 'TERORIS': 78, 'FPISumberKeributan': 75, 'papualawanhoax': 74, 'FPIPatutDibasmi': 71, 'papuaindonesia': 71, 'KamtibmasCiptaKondusif': 70, 'PidatoPresiden2021': 69, 'PolriPON2021Aman': 68, 'VaksinasiPerketatProkes': 68, 'bijakbermedsos': 68, 'NewRadikalisme2021': 65, 'Covid19': 62, 'SatgasNusantara': 60, 'PPKMLonggarProkesJalan': 59, 'PolriPeduliAnak': 59, 'HimbauanIsolasiTerpusat': 58, 'PolriPastikanProkes': 58, 'PerketatDisiplinProkes': 55, 'PercepatKekebalanKomunal': 55, 'PolriPastikanAman': 55, 'KKBPAPUA': 53, 'Vaksinasi

In [9]:
# converts dictionary to dataframe
df_lst = []
for d in sorted_dict_lst:
    df_lst.append(pd.DataFrame(d.items(), columns=["hashtag", "count"]))

In [10]:
## Downloads data to local
i = 0
name = ['TMRC14_APAC_1', 'TMRC14_APAC_2', 'TMRC15_APAC_3']
for account_df in df_lst:
    account_df.to_csv(f"~/Coding/buntain/hashtag_data/{name[i]}.csv", index=False)
    i+=1
